In [1]:
import torch
import numpy as np
import hvplot.pandas  # Import hvplot for pandas
import pandas as pd
import panel as pn
import holoviews as hv  # Import Holoviews
import xarray as xr  # Import xarray for creating a labeled multi-dimensional array

# Hardcoded parameters
n_shots = 16
n_receivers = 100
n_time_samples = 300
noise_std_dev = 0.1  # Standard deviation of Gaussian noise
central_frequency = 10.0  # Frequency of Ricker wavelet

In [9]:
# Function to generate a Ricker wavelet
def ricker(frequency, length, dt):
    t = np.arange(0, length) * dt - (length * dt / 2)
    y = (1 - 2 * (np.pi**2) * (frequency**2) * (t**2)) * np.exp(
        -(np.pi**2) * (frequency**2) * (t**2)
    )
    return y

# Generate base data
wavelet = torch.tensor(
    ricker(central_frequency, n_time_samples, 1 / n_time_samples)
)
obs_data = wavelet[None, None, :] + noise_std_dev * torch.randn(
    n_shots, n_receivers, n_time_samples
)

# Convert to DataFrame for hvPlot
data_frame = pd.DataFrame(obs_data.numpy().reshape(-1, n_time_samples))

In [10]:
def plot_1D(shot_index, receiver_index):
    return data_frame.iloc[shot_index * n_receivers + receiver_index].hvplot()

def plot_2D(shot_index):
    # Convert the selected shot data to an xarray DataArray
    shot_data = xr.DataArray(
        obs_data[shot_index].numpy(),
        dims=("receiver", "time"),
        coords={"receiver": range(n_receivers), "time": range(n_time_samples)}
    )
    return hv.Image(shot_data, kdims=['receiver', 'time']).opts(cmap='viridis', colorbar=True, width=600, height=400)

In [11]:
dim_selector = pn.widgets.RadioBoxGroup(
    name='Dimension', options=['1D', '2D'], inline=True
)
shot_slider = pn.widgets.IntSlider(name='Shot Index', start=0, end=n_shots - 1)
receiver_slider = pn.widgets.IntSlider(
    name='Receiver Index', start=0, end=n_receivers - 1
)


@pn.depends(
    dim_selector.param.value,
    shot_slider.param.value,
    receiver_slider.param.value,
)
def reactive_plot(dim, shot_idx, receiver_idx):
    if dim == '1D':
        return plot_1D(shot_idx, receiver_idx)
    else:
        return plot_2D(shot_idx)

In [12]:
layout = pn.Row(
    pn.Column(dim_selector, shot_slider, receiver_slider), reactive_plot
).servable()

layout

/home/tyler/anaconda3/envs/dw/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/tyler/anaconda3/envs/dw/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


Row
    [0] Column
        [0] RadioBoxGroup(inline=True, name='Dimension', options=['1D', '2D'], value='1D')
        [1] IntSlider(end=15, name='Shot Index')
        [2] IntSlider(end=99, name='Receiver Index')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)